# Exercises

## Acquire (`acquire.py`)

### Zillow
**For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.**

**You will have a** 
`zillow.ipynb` 
**file and a helper file for each section in the pipeline.**

In [1]:
###############################################################################
### python imports                                                          ###
###############################################################################

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LassoCV

import warnings
warnings.filterwarnings("ignore")

# from math import sqrt
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import acquire as acq
import prepare as prep

from debug import local_settings, timeifdebug, timeargsifdebug, frame_splain
from dfo import DFO, set_dfo

In [3]:
local_settings.splain=True
local_settings.debug=True
splain = local_settings.splain
debug = local_settings.debug

In [4]:
from wrangle_zillow import get_zillow_data, prep_zillow_data, sql

#### Acquire & Summarize

1. **Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.**

   - *Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for* 
`airconditioningtypeid`
*.*
   - Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
   - Only include properties that include a latitude and longitude value.

In [5]:
dfo = set_dfo(get_zillow_data(), splain=True)

starting get_zillow_data
starting sql_df
starting get_db_url
ending get_db_url ; time: 4.76837158203125e-06
starting check_df
starting frame_splain
DATAFRAME SHAPE:
(77381, 68) 

DATAFRAME INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
c

2. **Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.).**

In [6]:
df = dfo.df

In [7]:
frame_splain(dfo.df, title='Zillow Data', splain=True)

starting frame_splain
ZILLOW DATA SHAPE:
(77381, 68) 

ZILLOW DATA INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null float64
finishedfloor1s

In [8]:
@timeifdebug
def convert_to_dates(df, cols=[], format='%Y-%m-%d', errors='coerce', **kwargs):
    '''
    convert_to_dates(
        df, 
        cols, 
        format='%y-%m-%d', 
        errors='coerce', 
        **kwargs
    )
    RETURNS df with columns updated to dates
    '''
    convertcols = [col for col in cols if col in df.columns]
    for col in convertcols:
        df[col] = pd.to_datetime(df[col], errors=errors, format=format)
    return df

In [9]:
df = convert_to_dates(df, cols=['transactiondate'])

starting convert_to_dates
ending convert_to_dates ; time: 0.10535502433776855


In [10]:
dfo.df.head(15)

,id,parcelid,airconditioningtypeid,airconditioningdesc,architecturalstyletypeid,architecturalstyledesc,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,...,taxamount,taxdelinquencyflag,taxdelinquencyyear,typeconstructiontypeid,typeconstructiondesc,censustractandblock,transactiondate,logerror,transactions,tax_rate
0,1387261,17052889,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,...,5672.48,NaN,NaN,NaN,NaN,6.111001e+13,2017-01-01,0.055619,1,0.012225
1,1447245,17143294,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,...,3508.10,NaN,NaN,NaN,NaN,6.111005e+13,2017-01-01,-0.020526,1,0.010596
2,43675,17110996,NaN,NaN,NaN,NaN,NaN,2.5,3.0,NaN,...,2204.84,NaN,NaN,NaN,NaN,6.111005e+13,2017-01-02,0.008669,1,0.011133
3,1327940,17153340,NaN,NaN,NaN,NaN,NaN,3.5,2.0,NaN,...,12738.54,NaN,NaN,NaN,NaN,6.111004e+13,2017-01-02,-0.095340,1,0.012201
4,1431120,17153706,NaN,NaN,NaN,NaN,NaN,2.5,2.0,NaN,...,8442.56,NaN,NaN,NaN,NaN,6.111004e+13,2017-01-02,0.027817,1,0.014142
5,1969702,17229300,NaN,NaN,NaN,NaN,NaN,2.5,2.0,NaN,...,2639.36,NaN,NaN,NaN,NaN,6.111008e+13,2017-01-02,-0.017684,1,0.010968
6,2490820,17134185,NaN,NaN,NaN,NaN,NaN,2.0,4.0,NaN,...,4557.52,NaN,NaN,NaN,NaN,6.111006e+13,2017-01-03,0.057690,1,0.010838
7,269618,17292247,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,...,1450.06,NaN,NaN,NaN,NaN,6.111006e+13,2017-01-03,-0.421908,1,0.018693
8,74982,17141654,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,...,4139.18,NaN,NaN,NaN,NaN,6.111005e+13,2017-01-03,-0.021898,1,0.010678
9,1894799,17148798,NaN,NaN,NaN,NaN,NaN,2.0,3.0,NaN,...,2409.36,NaN,NaN,NaN,NaN,6.111009e+13,2017-01-03,-0.046017,1,0.011645


In [11]:
frame_splain(dfo.df, splain=True)

starting frame_splain
DATAFRAME SHAPE:
(77381, 68) 

DATAFRAME INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null object
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null object
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null object
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squar

3: **Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.**

||num_rows_missing|pct_rows_missing|
|:---|---:|---:|
|parcelid|0|0.000000|
|airconditioningtypeid|29041|0.535486|
|architecturalstyletypeid|54232|0.999982|


In [12]:
@timeifdebug
def get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='',
        limit_to_max=False,
        **kwargs
        ):
    '''
    get_column_values_stats(
        df, 
        get_cols=[], 
        max_uniques=10, 
        target_col='', 
        limit_to_max=False,
        **kwargs
        )
    RETURNS summary dataframe

    Receives dataframe as input, examines all columns defined as objects, and
    returns a summary report with column name as its index.

    Report shows the following information about the columns of the dataframe:
        cols: column name
        dtypes: data type
        num_rows_values: number of rows with non-null values 
        num_rows_missing: number of rows with missing values
        pct_rows_missing: percentage of rows with missing values
        num_uniques: number of unique values 
        unique_values: list of unique values if the unique value count is less than 
            or equal to max_uniques.
        
    If the input dataframe contains the target column, enter that name as the 
    target_col argument and it will be removed from the analysis.
    
    If limit_to_max is True (default), the resulting dataframe will only show
    columns with no more unique values than specified in max_uniques.
    
    NOTE: This function can be used to identify and then re-type catergorical
    columns. Run the function with the following parameters, change the data 
    types of know categorical values to 'category', then re-run this function, 
    gradually increasing the max_uniques value until the desired limit for 
    categoricals has been met.
    
    get_column_values_stats(
        df, 
        get_cols=df.columns[df.dtypes!='category'],
        max_uniques=5,
        limit_to_max=True
        )
    '''
    # get rowcount
    num_recs = len(df)
    
    # if no columns presented, get all columns from df
    use_cols = df.columns if len(get_cols) == 0 else get_cols
    # ensure use_cols are actually present in df
    cols = [col for col in use_cols if col in df.columns]
    
    # make df for all columns, exclude target column if passed
    df_cols = pd.DataFrame(cols, columns=['cols'])
    df_cols = df_cols[df_cols.cols != target_col]
    
    # get statistics on all columns
    df_cols['dtype'] = df_cols.cols.apply(lambda x: df[x].dtype)
    df_cols['num_rows_values'] = df_cols.cols.apply(lambda x: df[x].count())
    df_cols['num_rows_missing'] = num_recs - df_cols.num_rows_values
    df_cols['pct_rows_missing'] = df_cols.num_rows_missing / num_recs
    df_cols['num_uniques'] = df_cols.cols.apply(lambda x: df[x].nunique())
    
    # get unique valeues for columns within unique value limits
    df_cats = df_cols[df_cols.num_uniques <= max_uniques]
    df_cats['unique_values'] = df_cats.cols.apply(lambda x: df[x].unique())
    
    # merge and set index
    join_type = 'inner' if limit_to_max else 'left'
    df_cols = df_cols.join(df_cats.unique_values, how=join_type)
    return df_cols.set_index('cols')

**Find string categoricals**

In [13]:
df_strings = get_column_values_stats(
    df, 
    get_cols=df.columns[df.dtypes=='object'],
    max_uniques=20000,
    limit_to_max=True
)
df_strings

starting get_column_values_stats
ending get_column_values_stats ; time: 0.5404069423675537


,dtype,num_rows_values,num_rows_missing,pct_rows_missing,num_uniques,unique_values
cols,,,,,,
airconditioningdesc,object,24953,52428,0.677531,5,"[nan, Central, Yes, None, Wall Unit, Refrigera..."
architecturalstyledesc,object,206,77175,0.997338,5,"[nan, Contemporary, Conventional, Ranch/Ramble..."
buildingclassdesc,object,15,77366,0.999806,2,"[nan, Buildings having wood or wood and steel ..."
county,object,77381,0,0.000000,3,"[Ventura, Los Angeles, Orange]"
state,object,77381,0,0.000000,1,[CA]
heatingorsystemdesc,object,49440,27941,0.361083,10,"[nan, Central, Floor/Wall, Forced air, Yes, No..."
propertycountylandusecode,object,77381,0,0.000000,75,"[1110, 1129, 1111, 1321, 1410, 0101, 010C, 010..."
propertylandusedesc,object,77381,0,0.000000,13,"[Single Family Residential, Condominium, Tripl..."
propertyzoningdesc,object,50331,27050,0.349569,1907,"[nan, LCR110000*, LAR3, PSR2, GLR4YY, WHRE2000..."


In [14]:
df_strings.index

Index(['airconditioningdesc', 'architecturalstyledesc', 'buildingclassdesc',
       'county', 'state', 'heatingorsystemdesc', 'propertycountylandusecode',
       'propertylandusedesc', 'propertyzoningdesc', 'storydesc',
       'taxdelinquencyflag', 'typeconstructiondesc'],
      dtype='object', name='cols')

In [15]:
string_categoricals = [
    'airconditioningdesc', 
    'architecturalstyledesc', 
    'buildingclassdesc', 
    'county', 
    'state', 
    'heatingorsystemdesc', 
    'propertycountylandusecode', 
    'propertylandusedesc', 
    'propertyzoningdesc', 
    'storydesc', 
    'taxdelinquencyflag', 
    'typeconstructiondesc'
]

In [16]:
df = prep.retype_cols(df, cols=string_categoricals, to_dtype='category')
df.info()

starting retype_cols
ending retype_cols ; time: 0.3244800567626953
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null float64
airconditioningdesc             24953 non-null category
architecturalstyletypeid        206 non-null float64
architecturalstyledesc          206 non-null category
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null float64
buildingclassdesc               15 non-null category
buildingqualitytypeid           49672 non-null float64
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null float64
finishedfloor1s

**Find Numeric Categoricals**

In [17]:
df_cat_check = get_column_values_stats(
    df, 
    get_cols=df.columns[df.dtypes!='category'],
    max_uniques=750000,
    limit_to_max=True
)
df_cat_check

starting get_column_values_stats
ending get_column_values_stats ; time: 0.45628833770751953


,dtype,num_rows_values,num_rows_missing,pct_rows_missing,num_uniques,unique_values
cols,,,,,,
id,int64,77381,0,0.000000,77381,"[1387261, 1447245, 43675, 1327940, 1431120, 19..."
parcelid,int64,77381,0,0.000000,77381,"[17052889, 17143294, 17110996, 17153340, 17153..."
airconditioningtypeid,float64,24953,52428,0.677531,5,"[nan, 1.0, 13.0, 5.0, 11.0, 9.0]"
architecturalstyletypeid,float64,206,77175,0.997338,5,"[nan, 7.0, 8.0, 21.0, 3.0, 2.0]"
basementsqft,float64,50,77331,0.999354,43,"[nan, 900.0, 1416.0, 224.0, 100.0, 512.0, 380...."
bathroomcnt,float64,77381,0,0.000000,22,"[1.0, 2.0, 2.5, 3.5, 3.0, 1.5, 0.0, 4.5, 6.0, ..."
bedroomcnt,float64,77381,0,0.000000,16,"[2.0, 3.0, 4.0, 0.0, 5.0, 6.0, 1.0, 7.0, 8.0, ..."
buildingclasstypeid,float64,15,77366,0.999806,2,"[nan, 4.0, 3.0]"
buildingqualitytypeid,float64,49672,27709,0.358085,12,"[nan, 8.0, 9.0, 5.0, 4.0, 6.0, 7.0, 10.0, 3.0,..."


In [18]:
df_cat_check.index

Index(['id', 'parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'decktypeid',
       'finishedfloor1squarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'propertylandusetypeid', 'rawcensustractandblock',
       'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip',
       'roomcnt', 'storytypeid', 'taxvaluedollarcnt', 'threequarterbathnbr',
       'unitcnt', 'yardbuildingsqft17', 'yardbuildingsqft26', 'yearbuilt',
       'numberofstories', 'fireplaceflag', 'structuretaxvaluedollarcnt',
       'assessmentyear

In [19]:
numeric_categoricals = [
    'airconditioningtypeid', 
    'architecturalstyletypeid', 
    'buildingclasstypeid', 
    'buildingqualitytypeid',
    'decktypeid', 
    'fips', 
    'hashottuborspa',
    'heatingorsystemtypeid',
    'pooltypeid10', 
    'pooltypeid2', 
    'pooltypeid7', 
    'propertylandusetypeid',
    'regionidcounty', 
    'regionidcity',
    'regionidneighborhood',
    'regionidzip',
    'rawcensustractandblock',
    'censustractandblock',
    'storytypeid', 
    'threequarterbathnbr', 
    'numberofstories', 
    'fireplaceflag', 
    'assessmentyear', 
    'typeconstructiontypeid', 
    'transactions'
]

In [20]:
df = prep.retype_cols(df, cols=numeric_categoricals, to_dtype='category')
df.info()

starting retype_cols
ending retype_cols ; time: 0.32583189010620117
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 68 columns):
id                              77381 non-null int64
parcelid                        77381 non-null int64
airconditioningtypeid           24953 non-null category
airconditioningdesc             24953 non-null category
architecturalstyletypeid        206 non-null category
architecturalstyledesc          206 non-null category
basementsqft                    50 non-null float64
bathroomcnt                     77381 non-null float64
bedroomcnt                      77381 non-null float64
buildingclasstypeid             15 non-null category
buildingclassdesc               15 non-null category
buildingqualitytypeid           49672 non-null category
calculatedbathnbr               76772 non-null float64
calculatedfinishedsquarefeet    77185 non-null float64
decktypeid                      614 non-null category
finishedf

4. **Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.**

num_cols_missing|pct_cols_missing|num_rows
:---|---:|---:
23|38.333|108
24|40.000|123
25|41.667|5280

In [21]:
def nulls_by_row(df):
    total_cols = df.shape[1]
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/ total_cols
    rows_missing = (
        pd.DataFrame(
        {
            'num_cols_missing': num_cols_missing, 
            'pct_cols_missing': pct_cols_missing
        }
        )
        .reset_index()
        .groupby(['num_cols_missing','pct_cols_missing'])
        .count()
        .rename(index=str, columns={'index': 'num_rows'})
        .reset_index()
    )
    return rows_missing 

In [22]:
nulls_by_row(df)

,num_cols_missing,pct_cols_missing,num_rows
0,20,0.29411764705882354,2
1,21,0.3088235294117647,11
2,22,0.3235294117647059,25
3,23,0.3382352941176471,63
4,24,0.35294117647058826,266
5,25,0.36764705882352944,529
6,26,0.38235294117647056,5408
7,27,0.39705882352941174,10221
8,28,0.4117647058823529,13116
9,29,0.4264705882352941,15827


#### Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2. Create a function that will drop rows or columns based on the percent of values that are missing: `handle_missing_values(df, prop_required_column, prop_required_row)`.

   - The input:
       - A dataframe
       - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
       - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
   - The output:
       - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
   - hint:
       - Look up the dropna documentation.
       - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
       - Make use of inplace, i.e. inplace=True/False.

In [23]:
def handle_missing_values(
    df, 
    prop_required_column, 
    prop_required_row
    ):
    '''
    handle_missing_values(
    df, 
    prop_required_column=.9, 
    prop_required_row=.9
    )
    '''
    

3. Decide how to handle the remaining missing values:
   - Fill with constant value.
   - Impute with mean, median, mode.
   - Drop row/column

In [27]:
df_sqft = df[['calculatedfinishedsquarefeet',
'finishedfloor1squarefeet',
'finishedsquarefeet6',
'finishedsquarefeet12',
'finishedsquarefeet13',
'finishedsquarefeet15',
'finishedsquarefeet50'
]]

#### `wrangle_zillow.py`

Functions of the work above needed to acquire and prepare a new sample of data.

In [28]:
df_sqft[df_sqft.calculatedfinishedsquarefeet.isna() & (df_sqft.finishedsquarefeet50.isna()==False)]

,calculatedfinishedsquarefeet,finishedfloor1squarefeet,finishedsquarefeet6,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50


In [26]:
df_pool = df[['poolcnt','poolsizesum','pooltypeid2','pooltypeid7','pooltypeid10']]
df_pool[(df_pool.poolcnt.isna()==False) | (df_pool.pooltypeid10.isna()==False)]

KeyError: "['pooltypeid2', 'pooltypeid10', 'poolsizesum', 'pooltypeid7'] not in index"